In [436]:
def arithmetic_coding_encode(input_string, frequency_dict, step=4):
    # Инициализируем начальные значения
    low = 0.0
    high = 1.0
    values_list = []

    i = 0
    # Проходимся по каждому символу в строке
    for symbol in input_string:

        # Обновляем границы интервала
        symbol_index = list(frequency_dict.keys()).index(symbol)
        new_low = low + (high - low) * (list(frequency_dict.values())[symbol_index - 1] if symbol_index != 0 else 0)
        new_high = low + (high - low) * list(frequency_dict.values())[symbol_index]

        # Обновляем нижнюю границу
        low = new_low

        # Обновляем верхнюю границу
        high = new_high

        i += 1
        if i == step or step * len(values_list) + i == len(input_string):
            values_list.append((low + high) / 2)
            low = 0.0
            high = 1.0
            i = 0

    return values_list

In [437]:
def arithmetic_coding_decode(encoded_values, frequency_dict, input_string_length, step=4):
    # Восстанавливаем исходную строку
    decoded_string = ""

    i = 0
    k = 0
    for _ in range(input_string_length):
        # Ищем символ в частотном словаре
        for symbol, symbol_high in frequency_dict.items():

            symbol_index = list(frequency_dict.keys()).index(symbol)
            symbol_low = list(frequency_dict.values())[symbol_index - 1] if symbol_index != 0 else 0
            # Если значение в пределах интервала для текущего символа
            if symbol_low <= encoded_values[k]+pow(10, -10) < symbol_high:
                decoded_string += symbol

                i += 1
                if i == step:
                    k += 1
                    i = 0
                else:
                    encoded_values[k] = (encoded_values[k] - symbol_low) / (symbol_high - symbol_low)

                break

    return decoded_string

In [134]:
from collections import Counter

def build_frequency_dict(text):
    frequency_dict = Counter(text)

    num_frequency_dict = {}
    point = 1 / sum(frequency_dict.values())
    num = 0
    for key in frequency_dict:
        num += point * frequency_dict[key]
        num_frequency_dict[key] = num

    return num_frequency_dict

In [396]:
import struct

def arithmetic_to_file(filename, encoded_text, dictionary, text_len):
    with open(filename, 'wb') as file:
        file.write(text_len.to_bytes(6, byteorder='big'))
        for item in encoded_text:
            file.write(bytearray(struct.pack("f", item)))
        for key in dictionary:
            file.write(ord(key).to_bytes(2, byteorder='big'))
            file.write(bytearray(struct.pack("f", dictionary[key])))
    return

In [438]:
def arithmetic_from_file(filename, step=4):
    with open(filename, 'rb') as file:
        text_len = int.from_bytes(file.read(6), byteorder='big')
        encoded_text = []
        for i in range(math.ceil(text_len / step)):
            encoded_text.append(struct.unpack("f", file.read(4))[0])
        
        dictionary = {}
        while True:
            tmp = file.read(2)
            if not tmp:
                break
            letter = chr(int.from_bytes(tmp, byteorder='big'))
            count = struct.unpack("f", file.read(4))[0]
            dictionary[letter] = count
    
    return encoded_text, dictionary, text_len

In [38]:
def read_from_file(filename):
    string = ''
    with open(filename, "r", encoding='utf-8') as file:
        for line in file:
            string += line
    return string

In [439]:
string = read_from_file('harry_potter_and_the_prisoner_of_azkaban_and_harry_potter_and_the_sorcerers_stone.txt')

In [440]:
dictionary = build_frequency_dict(string)
encode_text = arithmetic_coding_encode(string, dictionary)

In [432]:
len(dictionary)

77

In [441]:
arithmetic_to_file('encoded_harry_potter_and_the_prisoner_of_azkaban_and_harry_potter_and_the_sorcerers_stone.txt', encode_text, dictionary, len(string))

In [442]:
encoded_text, dictionary, text_len = arithmetic_from_file('encoded_harry_potter_and_the_prisoner_of_azkaban_and_harry_potter_and_the_sorcerers_stone.txt')

In [443]:
arithmetic_coding_decode(encoded_text, dictionary, text_len)

' \n  \n   Harry Potter and the Prisoner of Azfaban\n  \n  \n   To Jill Prewett and Aine Kiely,\n   The godmothers of Swing\n  \n  \n   \n    1. OWL PsST ?  \n   Harry Potter was a highly unusual boy in many ways. For one thing, he hated the summer holidays more than any other time of year. For another, he really wanted to do his homework but was forced to do it in secret, in the dead of night. And he also happened to be a wizard.\n   It was nearly midnight, and he was lying on his stomach in bed, the blankets drawn right over his head like a tent, a flashlight in one hand and a large leather bound book M  History of Magic by Bathilda Bagshot) propped open against the pillow. Harry moved the tip of his eagle feather quill down the page, frowning as he looked for something that would help him write his essay, "Witch Burning in the Fourteenth Century Was Comcletely Pointless Discuss."\n   The quill paused at the top of a likely looking paragraph. Harry pushed his round glasses up the bri

In [371]:
import math

def entropy(probabilities):
    entropy_value = 0.0
    for probability in probabilities:
        if probability != 0:
            entropy_value -= probability * math.log2(probability)
    return entropy_value

In [389]:
prob_list = [list(dictionary.values())[0]]
for i in range(1, len(dictionary), 1):
    prob_list.append(list(dictionary.values())[i] - list(dictionary.values())[i-1])

In [390]:
entropy(prob_list)

4.522107417749594